#Chatbot using NLP LancasterStemmer and tflearn tensorflow labrary

In this notebook, we will assemble a  LancasterStemmer model using tflearn Functional API to create a working Chatbot which would answer questions about covid 19

##1) Importing the packages
We will import TensorFlow and  tflearn and nltk. Also, we import other modules which help in defining model layers

In [ ]:
pip install tensorflow==1.15

In [ ]:
pip install tflearn 

     |████████████████████████████████| 102kB 1.5MB/s 
  Created wheel for tflearn: filename=tflearn-0.3.2-cp36-none-any.whl size=128207 sha256=862c5e10d9bcc87fc43711823d1780b8900f3c286ba7a316680a5189583d37a8
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [ ]:
import nltk 
import tensorflow
import numpy 
import tflearn 
import pickle 
import random 
import json 
nltk.download('punkt') 







[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## 2) Preprocessing the data

###A)Importing data in json format

 It contains pairs of questions and answers about coronavirus

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#from google.colab import files
#uploaded = files.upload()
data = json.loads(open('/content/gdrive/My Drive/DOCBOT PROJECT/instens.json').read())

### B) Reading and Preparing the data 

In [ ]:
from nltk.stem.lancaster import LancasterStemmer 
stemmer = LancasterStemmer() 


TFLearn supports numpy array data 

In [ ]:


try: 
    with open("data.pickle", "rb") as f: 
        words, l, training, output = pickle.load(f) 
except: 
      
    #  Extracting Data 
    words = [] 
    l = [] 
    docs_x = [] 
    docs_y = [] 
      
   # converting each pattern into list of words using nltk.word_tokenizer  
    for i in data["intents"]:    
        for p in i["patterns"]: 
            wrds = nltk.word_tokenize(p) 
            words.extend(wrds) 
            docs_x.append(wrds) 
            docs_y.append(i["tag"]) 
   
            if i["tag"] not in l: 
                l.append(i["tag"]) 
    # Word Stemming             
    words = [stemmer.stem(w.lower()) for w in words if w != "?"]          
    words = sorted(list(set(words))) 
    l = sorted(l)                                       
      
    # This code will simply create a unique list of stemmed  
    # words to use in the next step of our data preprocessing 
    training = [] 
    output = [] 
    out_empty = [0 for _ in range(len(l))] 
    for x, doc in enumerate(docs_x): 
        bag = [] 
   
        wrds = [stemmer.stem(w) for w in doc] 
   
        for w in words: 
            if w in wrds: 
                bag.append(1) 
            else: 
                bag.append(0) 
        output_row = out_empty[:] 
        output_row[l.index(docs_y[x])] = 1
   
        training.append(bag) 
        output.append(output_row) 
          
    # Finally we will convert our training data and output to numpy arrays     
    training = numpy.array(training)         
    output = numpy.array(output) 
    with open("/content/gdrive/My Drive/opensourceproject/data.pickle", "wb") as f: 
        pickle.dump((words, l, training, output), f) 
  

##3)Creating the layers of the neural network models in TFLearn andTraining the model  

TFLearn integrates a set of functions that can easily handle any neural network training, whatever the number of inputs, outputs and optimizers.

While using TFlearn layers, many parameters are already self managed, so it is very easy to train a model, using DNN model class:

In [ ]:
# Developing a Model         
tensorflow.reset_default_graph()                     
net = tflearn.input_data(shape=[None, len(training[0])]) 
net = tflearn.fully_connected(net, 8) 
net = tflearn.fully_connected(net, 8) 
net = tflearn.fully_connected(net, len(output[0]), activation="softmax") 
net = tflearn.regression(net)  
model = tflearn.DNN(net) 
'''try:                                
    model.load("model.tflearn") 
except:'''
# Training 
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)         

 


Training Step: 4999  | total loss: 0.33841 | time: 0.024s
| Adam | epoch: 1000 | loss: 0.33841 - acc: 0.9782 -- iter: 32/37
Training Step: 5000  | total loss: 0.31335 | time: 0.030s
| Adam | epoch: 1000 | loss: 0.31335 - acc: 0.9804 -- iter: 37/37
--


### Saving the our model

In [ ]:
model.save("/content/gdrive/My Drive/opensourceproject/model.tflearn") 

INFO:tensorflow:/content/gdrive/My Drive/opensourceproject/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
def bag_of_words(s, words):                                 
    bag = [0 for _ in range(len(words))] 
   
    s_words = nltk.word_tokenize(s) 
    s_words = [stemmer.stem(word.lower()) for word in s_words] 
   
    for se in s_words: 
        for i, w in enumerate(words): 
            if w == se: 
                bag[i] = 1
   
    return numpy.array(bag)

## 4) Talking with our Chatbot


In [ ]:
import string
from termcolor import colored


In [ ]:
def chat(): 
    print("""Start talking with the bot and ask your 
    queries about Corona-virus(type quit to stop)!""") 
      
    while True: 
        inp = input("You: ") 
        if inp.lower() == "quit": 
            break
   
        results = model.predict([bag_of_words(inp, words)])[0] 
        results_index = numpy.argmax(results) 
          
        #print(results_index) 
        tag = l[results_index] 
        if results[results_index] > 0.5: 
            for tg in data["intents"]: 
                if tg['tag'] == tag: 
                    responses = tg['responses'] 
   
           
            s=random.choice(responses)
            
            lines =s.split('.')
            print("docbot: ")
            for line in lines:
              text= colored(line,'green')

              
              print(text)
             
              
              
        else: 
            text1=colored('docbot: I am sorry but I can not understand','red')
            print(text1) 
            
   
chat() 

Start talking with the bot and ask your 
    queries about Corona-virus(type quit to stop)!
You: hello
docbot: 
Hello!
You: what is a coronavirus
docbot: 
Coronaviruses are a large family of viruses which may cause illness in animals or humans
  In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS)
 The most recently discovered coronavirus causes coronavirus disease COVID-19

You: What are the symptoms of COVID 19
docbot: 
The most common symptoms of COVID-19 are fever, tiredness, and dry cough
 Some patients may have aches and pains, nasal congestion, runny nose, sore throat or diarrhea
 These symptoms are usually mild and begin gradually
 Some people become infected but don’t develop any symptoms and don't feel unwell
 Most people (about 80%) recover from the disease without needing special treatment
 Around 1 out of ev